In [ ]:
###Run elastic seartch
# docker run -it \
#     --name elasticsearch \
#     -p 9200:9200 \
#     -p 9300:9300 \
#     -e "discovery.type=single-node" \
#     -e "xpack.security.enabled=false" \
#     docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [ ]:
#!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json

In [1]:
# #Parse the documents.json file
# import json

# with open('./documents.json', 'rt') as f_in:
#     documents_file = json.load(f_in)

# documents = []

# for course in documents_file:
#     course_name = course['course']

#     for doc in course['documents']:
#         doc['course'] = course_name
#         documents.append(doc)

In [3]:
# #Connect to elasticsearch
# from elasticsearch import Elasticsearch

# es = Elasticsearch("http://localhost:9200")
# es.info()

ObjectApiResponse({'name': 'a035e66779c4', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'iPkhqCf1Tcqwtxhyoc8Z_Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
#Create the index (basically like a SQL Table) in elastic search
# index_settings = {
#     "settings": {
#         "number_of_shards": 1,
#         "number_of_replicas": 0
#     },
#     "mappings": {
#         "properties": {
#             "text": {"type": "text"},
#             "section": {"type": "text"},
#             "question": {"type": "text"},
#             "course": {"type": "keyword"} 
#         }
#     }
# }

# index_name = "course-questions"
# response = es.indices.create(index=index_name, body=index_settings)

# response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [5]:
# #Put the files into elasticsearch
# from tqdm.auto import tqdm

# for doc in tqdm(documents):
#     es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [122]:
#Question3 Searching - 
def retrieve_documents_question3(query, index_name="course-questions", max_results=5):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    # documents = [hit['_source'] for hit in response['hits']['hits']]
    return response['hits']

In [123]:
user_question = "How do I execute a command in a running docker container?"

response = retrieve_documents_question3(user_question)

response

{'total': {'value': 865, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions',
   '_id': 'G9sSKJABgFjTYh86U0MJ',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions',
   '_id': 'KdsSKJABgFjTYh86S0Fc',
   '_score': 75.54128,
   '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork

In [124]:
###Question 4

In [125]:
def retrieve_documents_question4(query, index_name="course-questions", max_results=3):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    return response['hits']


In [126]:
user_question = "How do I execute a command in a running docker container?"

response = retrieve_documents_question4(user_question)

In [127]:
response

{'total': {'value': 345, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions',
   '_id': 'G9sSKJABgFjTYh86U0MJ',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions',
   '_id': 'OtsSKJABgFjTYh86U0Ns',
   '_score': 51.04628,
   '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use 

In [ ]:
###Question 5

In [128]:
def retrieve_documents_question5(query, index_name="course-questions", max_results=3):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [132]:
user_question = "How do I execute a command in a running docker container?"

response = retrieve_documents_question5(user_question)
response


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [144]:
user_question = "How do I execute a command in a running docker container?"

response = retrieve_documents_question5(user_question)

context = ""

for doc in response:

    doc_str = f"Q:{doc['question']}\nA:{doc['text']}\n\n"
    context += doc_str

context = context.strip()


In [145]:
def make_prompt(question, context):
    prompt_template = f"""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()
    return prompt_template

In [146]:
#Context Sanity Check
print(context)

Q:How do I debug a docker container?
A:Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q:How do I copy files from my local machine to docker container?
A:You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q:How do I copy files from a different folder into docker container’s working directory?
A:You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:

In [147]:
prompt = make_prompt("How do I execute a command in a running docker container?", context=context)

In [148]:
len(prompt)

1472

In [103]:
context

'Question: How do I debug a docker container?\nAnswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQuestion: How do I copy files from my local machine to docker container?\nAnswer: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQuestion: How do I copy files from a different folder into docker container’s working directory?\nAnswer: You can copy files from your local machine into a Docker 

In [143]:
#Question 6
import tiktoken
encoding = tiktoken.encoding_for_model('gpt-4o')

In [151]:
tokens = encoding.encode(prompt)

In [153]:
len(tokens)

326

In [154]:
tokens[:5]

[63842, 261, 4165, 14029, 29186]

In [156]:
encoding.decode(tokens[:5])

"You're a course teaching assistant"

In [157]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [158]:
import ollama
response = ollama.chat(model='phi3', messages=[
    {"role": "system", "content": context},
    {"role": "user", "content": prompt}
#   {'role': 'user','content': f"{prompt}", },
])
print(response['message']['content'])

 If the container is already running, execute a command in the specific container by using the `docker exec` command with the `-it` flags for interactive mode and specifying the container ID:

```bash
docker exec -it <container-id> bash
```

Replace `<container-id>` with the actual identifier of your running docker container. This will allow you to execute commands within the container's environment.


In [ ]:
#https://github.com/mtayyab2/Chat_with_PDF/blob/main/chat_with_pdf.py